<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 遷移學習
### 記得要用GPU的ram來跑深度學習
大公司有超強大的算力，才能訓練出來那麼好的模型，我們就是拿來用

content資料夾的drive, 跟sample_data同一層，所以不用寫絕對路徑\
keras應用 （應用模型、權重）\
[ https://keras.io/zh/applications/ ]


VGG16(當初訓練時,是用caffe, 所以 x的欄位 input/ouput資料不要 ÷255)\
[ https://keras.io/zh/applications/#vgg16_1 ]

zipfile 壓縮/解壓文件的工具\
[ https://docs.python.org/zh-cn/3/library/zipfile.html ]

dropout的原因\
[ https://zhuanlan.zhihu.com/p/38200980 ]

In [ ]:
import zipfile
path = "drive/MyDrive/GAN/train.zip"
f = zipfile.ZipFile(path)
# 解壓縮後放跟sample_data同一層，所以斷線後解壓縮後圖片檔會消失，重新執行即可
f.extractall()

In [ ]:
import glob
from PIL import Image
fs = glob.glob("train/*")
print(len(fs))
Image.open(fs[5000]).resize((224, 224)).convert("RGB")

In [ ]:
import pandas as pd
dog = glob.glob("train/dog.*")
cat = glob.glob("train/cat.*")
datas = pd.DataFrame({
    "path":dog+cat,
    "ans":[0]* len(dog) + [1]* len(cat)
})
print(len(dog))
print(len(cat))
print(datas)
print(datas.shape)

In [ ]:
# 遷移學習, 偷它訓練好的weights
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

# 下游任務

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
vgg_layers = vgg.layers
print(len(vgg_layers))
# 減少不用訓練的參數(原本模型已經幫我訓練好了)
for l in vgg_layers:
    l.trainable = False
clf_layers = [
        # 原本的平均及標準差各512個, 新的平均＆標準差也各512個=> 512*2*2=2048
        # 原本的平均＆標準差不做訓練:Non-trainable + 1024, 新的平均＆標準差: trainable + 1024
        BatchNormalization(),
        Flatten(),
        # 遷移學習下, 且資料量不夠大量,(圖片量)不足以支撐那麼大的模型(參數總數), 就選小一點,少一點神經元
        Dense(128, activation="relu"),
        Dropout(0.25),
        Dense(2, activation="softmax")
]
model = Sequential(vgg_layers + clf_layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
# demo pre: 你的預處理也要抄當初模型訓練時的預處理 (x_train的圖片不要 ÷255.0 )
# [ https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py ]
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
img = Image.open(fs[5000]).resize((224, 224)).convert("RGB")
img_np = np.array(img)
print(img_np)
preprocess_input(img_np)

numpy的random
[ https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html ]

In [ ]:
x, y = np.array(datas["path"]), np.array(datas["ans"])
print(len(x), len(y))

In [ ]:
# 0~3 隨機整數的 2維矩陣(想幾維都可以)
# np.random.randint(0, 4, size=(2, 3))
def get_data(x, y, batch=20):
    batch = 20
    idx = np.random.randint(0, len(x), size=batch)
    print(type(idx), len(idx))
    y_batch = y[idx]
    # print(y_batch)
    x_batch, x_batch_ori = [], []
    for path in x[idx]:
        img = Image.open(path).resize((224, 224)).convert("RGB")
        img_np = np.array(img)
        # print(img_np)
        img_np_pre = preprocess_input(img_np)
        x_batch.append(img_np_pre)
        x_batch_ori.append(img_np)
    return (np.array(x_batch), np.array(y_batch), np.array(x_batch_ori))
x_batch, y_batch, x_batch_ori = get_data(x, y)
print(x_batch.shape)
print(y_batch.shape)
print(x_batch_ori.shape)

train_on_batch [ https://keras.io/zh/models/model/ ]

In [ ]:
for i in range(60):
    print("-"*20, i, "-"*20)
    x_batch, y_batch, x_batch_ori = get_data(x, y)
    # 自己訓練,自己驗證
    result = model.train_on_batch(x_batch, y_batch)
    print("[Train]:", result)
    x_batch, y_batch, x_batch_ori = get_data(x, y)
    result = model.test_on_batch(x_batch, y_batch)
    print("[Validate]:", result)

Question1: 出現loss = 0.0 (訓練時出現完全的0、1)

Question2: train的loss 從 0.0突然下次又跳到2.2多 (訓練的loss誇張的忽高忽低)

Question3: predict時,貓或狗的機率有出現完全1 or 0 (不如何機率的函數) \
---->太常出現,所以模型有問題


In [ ]:
x_batch, y_batch, x_batch_ori = get_data(x, y, 2000)
model.evaluate(x_batch, y_batch)

In [ ]:
model.predict(x_batch)

In [ ]:
vgg.layers[1].get_weights()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
pre = model.predict_classes(x_batch)
trans = ["Dog", "Cat"]
idx = np.nonzero(pre != y_batch)[0][:200]
# print(np.nonzero(pre != y_test_r)[0].size)
false_img = x_batch_ori[idx]
false_pre = pre[idx]
false_label = y_batch[idx]
plt.figure(figsize=(14, 14))
width = 10
height = len(idx) // width + 1
for i in range(len(idx)):
    plt.subplot(height, width, i+1)
    plt.imshow(false_img[i])
    t = "[Ori]:{}\n[Pre]:{}".format(trans[false_label[i]], trans[false_pre[i]])
    plt.title(t)
    plt.axis("off")


In [ ]:
from PIL import Image
import requests
headers ={
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"
}
url = input("url: ")
response = requests.get(url, headers=headers, stream=True, verify=False)
# 要跟訓練資料的圖片相同大小、相同色碼
img = Image.open(response.raw).resize((224, 224)).convert("RGB")
img_np = np.array(img)
# 要與訓練同維度，調整圖片為四個維度(1筆資料, 長32, 寬32, 色碼3)
img_np_r = preprocess_input(img_np.reshape(1, 224, 224, 3))
proba = model.predict(img_np_r)[0]
for i, p in enumerate(proba):
    print(trans[i], "的機率:", p)
ans = model.predict_classes(img_np_r)[0]
print("答案:", trans[ans])
plt.imshow(img_np)